# download model

In [ ]:

!wget https://bapengstorage.blob.core.windows.net/fileshare/GODEL-XLarge.tar.gz
!tar -zxvf GODEL-XLarge.tar.gz

--2022-06-28 06:44:41--  https://bapengstorage.blob.core.windows.net/fileshare/GODEL-XLarge.tar.gz
Resolving bapengstorage.blob.core.windows.net (bapengstorage.blob.core.windows.net)... 13.66.176.16
Connecting to bapengstorage.blob.core.windows.net (bapengstorage.blob.core.windows.net)|13.66.176.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19643503525 (18G) [application/gzip]
Saving to: ‘GODEL-XLarge.tar.gz’

GODEL-XLarge.tar.gz 100%[===================>]  18.29G  23.6MB/s    in 11m 20s 

2022-06-28 06:56:01 (27.6 MB/s) - ‘GODEL-XLarge.tar.gz’ saved [19643503525/19643503525]



# install libs

In [ ]:
# installs
!pip install transformers
!pip install dotmap

# inference

In [ ]:
import torch
import numpy as np
import dotmap

from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoModelForCausalLM
)


def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


model = None
tokenizer = None
args = dotmap.DotMap()
args.model_name_or_path = 'GODEL-XLarge'
args.prompt = ''
args.padding_text = ''
args.length = 128
args.num_samples = 1
args.temperature = 1
args.num_beams = 5
args.repetition_penalty = 1
args.top_k = 0
args.top_p = 0.5
args.no_cuda = False
args.seed = 2022
args.stop_token = '<|endoftext|>'
args.n_gpu = 1
args.device = 'cuda:5'

set_seed(args)

#config = AutoConfig.from_pretrained(args.model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path)

model = model.half().to(args.device)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)




In [ ]:


    


def generate(context, knowledge):
    
    
    
    context = ' EOS '.join(context)

    input_ids = tokenizer(context + ' <|knowledge|> ' + knowledge +
                          ' =>', return_tensors="pt").input_ids.to(args.device)
    gen_kwargs = {
        # 'num_beams': args.num_beams,
        'max_length': args.length,
        'min_length': 32,
        'top_k': 2000,
        'no_repeat_ngram_size': 4

    }

    output_sequences = model.generate(input_ids, **gen_kwargs)
    output_sequences = tokenizer.batch_decode(
        output_sequences, skip_special_tokens=True)

    return output_sequences[0].split('=>')[1]

In [ ]:
context = ['User: Hi']
knowledge = 'I am searching for new knoweledge.'
generate(context,knowledge )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Do you have any interesting things to tell me?'